# Computer Vision: scikit-image + scikit-learn

![](python-logo.png)
![](logo.png)
![](scikit-learn-logo-small.png)

# Problema: 
## Classificar Imagens de Motociclistas
#### *Com capacete* 

#### *Sem capacete*


In [3]:
import numpy as np
from numpy import copy
import matplotlib.pyplot as plt
from skimage.io import imread_collection,imsave
from sklearn.model_selection import train_test_split
from glob import glob
from scipy.stats import randint as sp_randint
import time
from skimage.color import rgb2grey
from skimage.filters import threshold_otsu
from skimage.measure import label,regionprops
import copy
from scipy.stats.mstats_basic import skew, kurtosis

%matplotlib inline

# Read images from database

In [4]:
path = "/Users/romuere/Dropbox/MTAP/databases/database2_raw_data/"

In [5]:
comcapacete = glob(path+'comcapacete/*.png')

In [6]:
len(comcapacete)

152

In [7]:
semcapacete = glob(path+'semcapacete/*.png')

In [8]:
images_cc = imread_collection(comcapacete)
images_sc = imread_collection(semcapacete)

# Segmentation

In [9]:
def segmentation(im):
    '''Recebe uma imagem calcula limiar de otsu e fazer o 
       recorte obdecendo a região resultante desse limiar'''
    
    grey_image = rgb2grey(im)
    otsu = threshold_otsu(grey_image)
    im_ = grey_image < otsu 
    
    n_branco = np.sum(im_ == 1)
    n_preto = np.sum(im_ == 0)
    if n_branco > n_preto:
        im_ = 1-im_
    
    label_img = label(im_, connectivity = grey_image.ndim)#detecta regioes nao conectadas
    props = regionprops(label_img)#calcula propriedade importantes de cada regiao encontrada (ex. area)

    #Convert todas as regioes que possuem um valor de area menor que a maior area em background da imagem 
    area = np.asarray([props[i].area for i in range(len(props))])#area de cada regiao encontrada
    max_index = np.argmax(area)#index da maior regiao
    for i in range(len(props)):
        if(props[i].area < props[max_index].area):
            label_img[np.where(label_img == i+1)] = 0#regiao menor que a maior eh marcada como background
 
    #----------------recorte da regiao de interesse----------------#
    # Obtendo os limites verticais das imagens segmentadas 
    ymin = np.min(np.where(label_img != 0)[1])
    ymax = np.max(np.where(label_img != 0)[1])
    imagem_cortada = imagem[:,ymin:ymax,:]     
    return imagem_cortada

In [10]:
start = time.time()
for id_im,imagem in enumerate(images_cc):
    im_name = images_cc.files[id_im].split('/')[-1]
    imagem_segmentada = segmentation(imagem)
    imsave(path+'segmentacao/comcapacete/'+im_name,imagem_segmentada)

    #print(im_name)
for id_im,imagem in enumerate(images_sc):
    im_name = images_sc.files[id_im].split('/')[-1]
    imagem_segmentada = segmentation(imagem)
    imsave(path+'segmentacao/semcapacete/'+im_name,imagem_segmentada)
    #print(im_name)
    
end = time.time()
print(end - start)

1.0888450145721436


In [11]:
labels = np.concatenate((np.zeros(len(comcapacete)),np.ones(len(semcapacete))))

# Extracting Features using GLCM

In [12]:
path_segmentada = "/Users/romuere/Dropbox/MTAP/databases/database2_raw_data/segmentacao/"
comcapacete = glob(path_segmentada+'comcapacete/*.png')
semcapacete = glob(path_segmentada+'semcapacete/*.png')
images = imread_collection(comcapacete+semcapacete)

In [13]:
from skimage.feature import greycomatrix,greycoprops

### Parameter

- Distance
- Angle

In [14]:
d = 15

In [15]:
features = np.zeros((len(labels),18)) #6 features x 3 color channels
start = time.time()

for id_im,imagem in enumerate(images):
    for id_ch in range(3):
        matrix0 = greycomatrix(imagem[:,:,id_ch], [d], [0],normed=True)
        matrix1 = greycomatrix(imagem[:,:,id_ch], [d], [np.pi/4],normed=True)
        matrix2 = greycomatrix(imagem[:,:,id_ch], [d], [np.pi/2],normed=True)
        matrix3 = greycomatrix(imagem[:,:,id_ch], [d], [3*np.pi/4],normed=True)
        matrix = (matrix0+matrix1+matrix2+matrix3)/4 
        props = np.zeros((6))
        props[0] = greycoprops(matrix,'contrast')
        props[1] = greycoprops(matrix,'dissimilarity')
        props[2] = greycoprops(matrix,'homogeneity')
        props[3] = greycoprops(matrix,'energy')
        props[4] = greycoprops(matrix,'correlation')
        props[5] = greycoprops(matrix,'ASM')
        features[id_im,id_ch*6:(id_ch+1)*6] = props

end = time.time()
print(end - start)

3.0643980503082275


# Split Data

In [16]:
train = 0.5
test = 1-train

In [17]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test)

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer, confusion_matrix,accuracy_score
from sklearn.svm import SVC


In [19]:
#Random Forest Parameter Estimation
def rf_parameter_estimation(xEst, yEst):
    
    clf = RandomForestClassifier(n_estimators=20)    
    # specify parameters and distributions to sample from
    hyperparameters = {"n_estimators": range(10,1000,50),
                  "max_depth": range(1,100),
                  "max_features": sp_randint(1, xEst.shape[1]),
                  "min_samples_split": sp_randint(1, xEst.shape[1]),
                  "min_samples_leaf": sp_randint(1, xEst.shape[1]),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    
    
    # run randomized search
    n_iter_search = 20
    random_search = RandomizedSearchCV(clf, param_distributions=hyperparameters,
                                       n_iter=n_iter_search,scoring=make_scorer(accuracy_score))
    
    
    random_search.fit(xEst, yEst)
    report(random_search.cv_results_)
    return random_search.best_params_

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))

# Classification unsing all features

In [20]:
start = time.time()

parameters = rf_parameter_estimation(X_train, y_train)
c_rf = RandomForestClassifier(**parameters)
c_rf.fit(X_train,y_train)
pred = c_rf.predict(X_test)
acc_rf = accuracy_score(y_test, pred)

end = time.time()
print(end - start)

Model with rank: 1
Mean validation score: 0.803 (std: 0.048)
Parameters: {'min_samples_split': 8, 'max_features': 17, 'n_estimators': 760, 'bootstrap': True, 'criterion': 'entropy', 'min_samples_leaf': 17, 'max_depth': 41}
Model with rank: 2
Mean validation score: 0.795 (std: 0.039)
Parameters: {'min_samples_split': 2, 'max_features': 10, 'n_estimators': 60, 'bootstrap': True, 'criterion': 'entropy', 'min_samples_leaf': 6, 'max_depth': 48}
Model with rank: 3
Mean validation score: 0.772 (std: 0.025)
Parameters: {'min_samples_split': 5, 'max_features': 4, 'n_estimators': 110, 'bootstrap': False, 'criterion': 'gini', 'min_samples_leaf': 1, 'max_depth': 41}
Model with rank: 3
Mean validation score: 0.772 (std: 0.020)
Parameters: {'min_samples_split': 16, 'max_features': 16, 'n_estimators': 60, 'bootstrap': False, 'criterion': 'entropy', 'min_samples_leaf': 3, 'max_depth': 1}
Model with rank: 3
Mean validation score: 0.772 (std: 0.020)
Parameters: {'min_samples_split': 15, 'max_features': 

In [21]:
print('Random Forest Accuracy: ',acc_rf)

Random Forest Accuracy:  0.7734375


# Classification using PCA

In [22]:
from sklearn.decomposition import PCA

In [23]:
components = [2,4,8,10,12]

In [24]:
def pca(X_train, X_test,y_train, n_comp):
    """
    PCA transformation for using a 'training' set and a 'testing' set
    """
    pca = PCA(n_components=n_comp)
    pca.fit(X_train,y_train)
    transform = pca.transform(X_test)
    return transform

In [25]:
results_rf = np.zeros(5)
results_svm = np.zeros(5)

start = time.time()
for id_comp,comp in enumerate(components):
    
    print('-------------','n comp. = ',comp,'-------------')
    
    X_train_pca = pca(X_train,X_train,y_train,comp)
    X_test_pca = pca(X_train,X_test,y_train,comp)
    #RF
    parameters = rf_parameter_estimation(X_train_pca, y_train)
    c_rf = RandomForestClassifier(**parameters)
    c_rf.fit(X_train_pca,y_train)
    pred = c_rf.predict(X_test_pca)
    acc = accuracy_score(y_test, pred)
    results_rf[id_comp] = acc
    
    
end = time.time()
print(end - start)

------------- n comp. =  2 -------------
Model with rank: 1
Mean validation score: 0.575 (std: 0.028)
Parameters: {'min_samples_split': 1, 'max_features': 1, 'n_estimators': 260, 'bootstrap': True, 'criterion': 'entropy', 'min_samples_leaf': 1, 'max_depth': 16}
Model with rank: 2
Mean validation score: 0.559 (std: 0.050)
Parameters: {'min_samples_split': 1, 'max_features': 1, 'n_estimators': 10, 'bootstrap': True, 'criterion': 'gini', 'min_samples_leaf': 1, 'max_depth': 35}
Model with rank: 2
Mean validation score: 0.559 (std: 0.050)
Parameters: {'min_samples_split': 1, 'max_features': 1, 'n_estimators': 160, 'bootstrap': True, 'criterion': 'gini', 'min_samples_leaf': 1, 'max_depth': 92}
Model with rank: 2
Mean validation score: 0.559 (std: 0.038)
Parameters: {'min_samples_split': 1, 'max_features': 1, 'n_estimators': 860, 'bootstrap': True, 'criterion': 'gini', 'min_samples_leaf': 1, 'max_depth': 65}
------------- n comp. =  4 -------------
Model with rank: 1
Mean validation score: 0.

In [26]:
plt.style.use('ggplot')
fig = plt.figure(figsize = (10,5),dpi=400)
ax = plt.subplot(111)
ax.plot(range(1,7),np.concatenate((results_rf,[acc_rf]),axis=0),marker = 'D',linestyle = ':',label = 'Random Forest')
ax.set_xlim([0,7])
ax.set_xlabel('PCA Components')
ax.set_ylabel('Accuracy')
ax.set_xticks(range(1,7))
ax.set_xticklabels(['2','4','8','10','12','No PCA'])
ax.set_ylim([0.5,1])
ax.set_title('Helmets')

In [42]:
fig.savefig('/Users/romuere/pca.png')